In [1]:
%reload_ext nb_black

<IPython.core.display.Javascript object>

## Day 77 Lecture 1 Assignment

In this assignment, we will learn about convolutions using Dask arrays.

In [2]:
import dask.array as da

<IPython.core.display.Javascript object>

In [3]:
import os
import random

import cv2
import numpy as np
import pandas as pd

<IPython.core.display.Javascript object>

In [4]:
image = cv2.imread("data/6-facial-recognition-hero.jpg", cv2.IMREAD_GRAYSCALE)

<IPython.core.display.Javascript object>

In [5]:
image.shape

(1360, 2040)

<IPython.core.display.Javascript object>

In [6]:
image_da = da.asarray(image)

<IPython.core.display.Javascript object>

In [29]:
image_da.compute()

array([[163, 163, 163, ..., 163, 163, 163],
       [163, 163, 163, ..., 163, 163, 163],
       [163, 163, 163, ..., 163, 163, 163],
       ...,
       [163, 163, 163, ..., 163, 163, 163],
       [163, 163, 163, ..., 163, 163, 163],
       [163, 163, 163, ..., 163, 163, 163]], dtype=uint8)

<IPython.core.display.Javascript object>

Rechunk the image array into chunks of 100x100.

In [7]:
# Answer below
chunked = da.rechunk(image_da, chunks=(100,100))


<IPython.core.display.Javascript object>

In [8]:
chunked.chunks

((100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 60),
 (100,
  100,
  100,
  100,
  100,
  100,
  100,
  100,
  100,
  100,
  100,
  100,
  100,
  100,
  100,
  100,
  100,
  100,
  100,
  100,
  40))

<IPython.core.display.Javascript object>

In [28]:
chunked.compute()

array([[163, 163, 163, ..., 163, 163, 163],
       [163, 163, 163, ..., 163, 163, 163],
       [163, 163, 163, ..., 163, 163, 163],
       ...,
       [163, 163, 163, ..., 163, 163, 163],
       [163, 163, 163, ..., 163, 163, 163],
       [163, 163, 163, ..., 163, 163, 163]], dtype=uint8)

<IPython.core.display.Javascript object>

In [24]:
min_array = chunked.min(axis=0)
max_array = chunked.max(axis=0)

<IPython.core.display.Javascript object>

In [25]:
min_array.compute()

array([163, 163, 163, ..., 163, 163, 163], dtype=uint8)

<IPython.core.display.Javascript object>

In [26]:
max_array.compute()

array([163, 163, 163, ..., 163, 163, 163], dtype=uint8)

<IPython.core.display.Javascript object>

In [27]:
scaled = (chunked - min_array) / (max_array - min_array)
scaled.compute()

C:\ProgramData\Anaconda3\lib\site-packages\dask\core.py:121: RuntimeWarning: invalid value encountered in true_divide
  return func(*(_execute_task(a, cache) for a in args))


array([[nan, nan, nan, ..., nan, nan, nan],
       [nan, nan, nan, ..., nan, nan, nan],
       [nan, nan, nan, ..., nan, nan, nan],
       ...,
       [nan, nan, nan, ..., nan, nan, nan],
       [nan, nan, nan, ..., nan, nan, nan],
       [nan, nan, nan, ..., nan, nan, nan]])

<IPython.core.display.Javascript object>

In [18]:
from sklearn.preprocessing import MinMaxScaler

scaler = MinMaxScaler()
scaler.fit(chunked)
image_scaled = scaler.transform(chunked)

image_scaled

C:\ProgramData\Anaconda3\lib\site-packages\dask\array\core.py:1395: FutureWarning: The `numpy.may_share_memory` function is not implemented by Dask array. You may want to use the da.map_blocks function or something similar to silence this warning. Your code may stop working in a future release.
  FutureWarning,


array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

<IPython.core.display.Javascript object>

In the code above, we read an image of a face and converted it into a Dask array. We will use this array to learn about convolutions. You can learn more about convolutions <a href="http://setosa.io/ev/image-kernels/">here</a>.

Convolutions are transformations of matrices that help us create convolutional neural network (which are crucial in deep learning) or apply effects to images like blurring or sharpening.

Our kernel is a 3x3 matrix which we apply to every 3x3 subset of our image matrix.

To start with our transformation, zero pad the image matrix with a width of one. This means that our output will be a matrix with an additional row and additional column of zeros on both sides. The image below illustrates padding. Use the pad function 
<img src="https://www.researchgate.net/profile/Terje_Midtbo/publication/318967374/figure/fig4/AS:669210783002634@1536563692382/Visualization-of-a-matrix-that-is-zero-padded-with-one-border.png">

In [45]:
# Answer below:

padded = da.pad(chunked, (1, 1), mode="edge")

<IPython.core.display.Javascript object>

What is the shape of our padded array?

In [46]:
# Answer below:
padded.shape


(1362, 2042)

<IPython.core.display.Javascript object>

To apply a convolution, we will install the `dask-image` package. Do this below.

In [47]:
# pip install dask-image

<IPython.core.display.Javascript object>

Import the convolve function from the package

In [48]:
from dask_image.ndfilters import convolve

<IPython.core.display.Javascript object>

Define the sharpen convolution from the setosa.io article mentioned above. Apply the convolution to our padded image.

In [49]:
# Answer below:
k = np.array([[0,-1,0],
             [-1,5,-1],
             [0,-1,0]])
filtered = convolve(padded, k, mode='constant')


<IPython.core.display.Javascript object>